# **Counting Objects Crossing the Line**
---
[![Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ayushexel/trolo/blob/master/recipes/count-objects-crossing-the-line.ipynb)

# Installation
---
#### Installing the Trolo package directly from the GitHub repo

In [ ]:
!pip install git+https://github.com/AyushExel/trolo.git

# Imports
---

In [ ]:
import numpy as np
from PIL import Image

import supervision as sv
from supervision.assets import download_assets, VideoAssets

In [ ]:
COCO_CLASSES = [
    "person", "bicycle", "car", "motorcycle", "airplane", "bus",
    "train", "truck", "boat", "traffic light", "fire hydrant", "stop sign",
    "parking meter", "bench", "bird", "cat", "dog", "horse", "sheep", "cow",
    "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella", "handbag",
    "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite",
    "baseball bat", "baseball glove", "skateboard", "surfboard", "tennis racket",
    "bottle", "wine glass", "cup", "fork", "knife", "spoon", "bowl", "banana",
    "apple", "sandwich", "orange", "broccoli", "carrot", "hot dog", "pizza",
    "donut", "cake", "chair", "couch", "potted plant", "bed", "dining table",
    "toilet", "tv", "laptop", "mouse", "remote", "keyboard", "cell phone",
    "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock",
    "vase", "scissors", "teddy bear", "hair drier", "toothbrush"
]

# Download Video
---
#### As an example, we will use a video from the `supervision.assets` library.

In [ ]:
import os
HOME = os.getcwd()

download_assets(VideoAssets.VEHICLES)

**NOTE**: To use your own input video, upload to colab and replace `SOURCE_VIDEO_PATH`.

In [ ]:
SOURCE_VIDEO_PATH = f"/{HOME}/vehicles.mp4"

In [ ]:
sv.VideoInfo.from_video_path(SOURCE_VIDEO_PATH)

# Inference
---

#### Initializes detection predictor with the specified Model

In [ ]:
from trolo.inference import DetectionPredictor

predictor = DetectionPredictor(model="dfine-n")

#### Initiales Annotations to be performed on Output Video

In [ ]:
bounding_box_annotator = sv.BoxAnnotator(thickness=4)
label_annotator = sv.LabelAnnotator(text_thickness=4, text_scale=2)
byte_tracker = sv.ByteTrack()

In [ ]:
START = sv.Point(0, 1500)
END = sv.Point(3840, 1500)

line_zone = sv.LineZone(start=START, end=END)

line_zone_annotator = sv.LineZoneAnnotator(
    thickness=4,
    text_thickness=4,
    text_scale=2)

# Running Object Detection
---
Using `sv.process_video`, we process the Video frame by frame and perform various annotations and final annotated video is saved at `TARGET_VIDEO_PATH`.

In [ ]:
def callback(frame: np.ndarray, index:int) -> np.ndarray:

    prediction = predictor.predict(Image.fromarray(frame))

    detections = sv.Detections(
        xyxy=prediction[0]['boxes'].numpy(),
        confidence=prediction[0]['scores'].numpy(),
        class_id=prediction[0]['labels'].numpy()
    )
    detections = byte_tracker.update_with_detections(detections)

    labels = [
        f"#{tracker_id} {COCO_CLASSES[class_id]} {confidence:0.2f}"
        for confidence, class_id, tracker_id
        in zip(detections.confidence, detections.class_id, detections.tracker_id)
    ]

    annotated_frame = frame.copy()
    annotated_frame = bounding_box_annotator.annotate(
        scene=annotated_frame,
        detections=detections)
    annotated_frame = label_annotator.annotate(
        scene=annotated_frame,
        detections=detections,
        labels=labels)

    line_zone.trigger(detections)

    return  line_zone_annotator.annotate(annotated_frame, line_counter=line_zone)

In [ ]:
TARGET_VIDEO_PATH = f'/{HOME}/output.mp4'

sv.process_video(
    source_path = SOURCE_VIDEO_PATH,
    target_path = TARGET_VIDEO_PATH,
    callback=callback
)